In [6]:
%load_ext autoreload
%autoreload 2

import get_info
from get_transcripts import get_html_data, get_transcripts
from process_transcript import Transcript
import pandas as pd 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'data_collection'

## Summary

This notebook collects the data to be displayed by the application.

1. A list of committees active during each term ==> `committees_list.json`
2. A detailed description of each committee ==> `committees_info.json`
3. Information on every sitting for every committee for every term ==> `committees_info.json`
4. A list of urls already tried ==> `urls_tried.csv`
5. Detailed transcripts of each sitting ==> ... 

In [3]:
# TEST: getting all committes active during the 10th term of the Sejm
# Contains basic information about each committee
committees_url = "https://api.sejm.gov.pl/sejm/term10/committees"
committees = get_info.get_json_data(committees_url)

# Save committees to a json file
get_info.write_json_to_file(committees, "data/committees.json")

# Structure of the output:
# Fields:
    # appointmentDate
    # code (acts as id)
    # compositionDate
    # members => list of dictionaries (club, function, id, lastFirstName)
    # name
    # nameGenitive
    # phone
    # scope
    # type

## Get parliamentary terms

```
{
    'current': False,
    'from': '1993-10-14',
    'num': 2,
    'prints': {'count': 0, 'link': '/term2/prints'},
    'to': '1997-10-19'},
}
```

In [106]:
terms = get_info.read_json_from_file("data/terms_info.json")

In [107]:
terms

[{'current': False,
  'from': '1993-10-14',
  'num': 2,
  'prints': {'count': 0, 'link': '/term2/prints'},
  'to': '1997-10-19'},
 {'current': False,
  'from': '1997-10-20',
  'num': 3,
  'prints': {'count': 0, 'link': '/term3/prints'},
  'to': '2001-10-18'},
 {'current': False,
  'from': '2001-10-19',
  'num': 4,
  'prints': {'count': 0, 'link': '/term4/prints'},
  'to': '2005-10-18'},
 {'current': False,
  'from': '2005-10-19',
  'num': 5,
  'prints': {'count': 0, 'link': '/term5/prints'},
  'to': '2007-11-04'},
 {'current': False,
  'from': '2007-11-05',
  'num': 6,
  'prints': {'count': 0, 'link': '/term6/prints'},
  'to': '2011-11-07'},
 {'current': False,
  'from': '2011-11-08',
  'num': 7,
  'prints': {'count': 4356,
   'lastChanged': '2024-06-18T03:34:50',
   'link': '/term7/prints'},
  'to': '2015-11-11'},
 {'current': False,
  'from': '2015-11-12',
  'num': 8,
  'prints': {'count': 4362,
   'lastChanged': '2024-05-18T03:35:28',
   'link': '/term8/prints'},
  'to': '2019-11-11

## Get basic info on committees

`committees_info` contains key information on committees such as:

- appointmentDate
- code
- compositionDate
- members [list of dicts: function, id, lastFirstName, club]
- name
- nameGenitive
- phone
- scope - short description of what the committee does
- type

In [132]:
# Adding term field to committees
committees_info = get_info.read_json_from_file("data/committees_info.json")
len(committees_info)

10

In [134]:

for i in range(1, len(committees_info)+1):
    print(i)
    for committee in committees_info[i-1]:
        committee["term"] = i
get_info.write_json_to_file(committees_info, "data/committees_info.json")

1
2
3
4
5
6
7
8
9
10


In [128]:
len(committees_info[9])
for c in committees_info[9]:
    print(c["code"])

ENM
FPB
GOR
ASW
INF
LPG
OBN
OSZ
PSR
RRW
ZDR
ESK
KFS
KSP
STR
SZA
SPC
UST
CNT
PET
NKK
GMZ
MNE
ODK
PSN
KOP
SKPG
RSP
KSS
EPS
SKGK
SUE
SKPC
NPC


## Get info on committee sittings

`sittings_info` includes basic information on each sitting in a following format:

```
{
    'agenda': '<div class="agenda-indent-0">Wybór Prezydium Komisji.</div>',
    'closed': False,
    'date': '2023-11-21',
    'num': 1,
    'remote': False,
    'video': ['DA71B7D8ED21825EC1258A6E003CB71B'],
    'code': 'ZDR',
    'term': < added by me = term number >,
    'code': < added by me = committee code >,
    'id': < added by me = term + committee code + sitting num >,
}
```

Code below produces an output which is a list of dicts with one dict for each committee sitting during a given term.

The data has a following structure:

```
{
    term1:
        { 
            committee_code_1: [list of dicts with sittings info ],
            committe_code_2: [{}, {}, ... ],
            ...
        },
    term2: 
        { 
            ...
        },
    ...
}
````

In [6]:
sittings_info = get_info.read_json_from_file("data/sittings_info.json")
# len(sittings) # 16 

# Fields:
    # agenda => items are packed in separate divs <div class="agenda-indent-0">item</div>\n
    # closed
    # date
    # num
    # remote
    # video

## Get transcripts

HTML files returned by the API seem to be well structured but divs with "stage directions" (who is speaking? who is present? is there a vote taking place?) are not marked with ids, names or classes. Speaker names and functions seem to be always in bold - this can be helpful. I will have to identify the speakers by crossrefencing the transcripts with a list of representatives / committee members. Since joint sittings happen occasionally the list of committee members may not be enough.

There is a `<section class="transcript">` which contains the transcript itself. The list of representatives present is not included in this section.

In [7]:
#del sittings_info["9"]
sittings_info["10"].keys()
del sittings_info["10"]["ODK"]
del sittings_info["10"]["STR"]
del sittings_info["10"]["SZA"]
del sittings_info["10"]["NPC"]
del sittings_info["10"]["ESK"]
del sittings_info["10"]["OBN"]
del sittings_info["10"]["SKPC"]
del sittings_info["10"]["ASW"]
del sittings_info["9"]
sittings_info["10"].keys()

dict_keys(['RRW', 'OSZ'])

In [8]:
# Let's get the transcripts
transcripts = get_transcripts(sittings_info)

10
RRW
OSZ


## Process transcript data

**Potential issues**

These bits are also included in the `<section class="transcript">`:
```
Komisja Administracji i Spraw Wewnętrznych, obradująca pod przewodnictwem wicemarszałka Sejmu Krzysztofa Bosaka, dokonała:

– wyboru prezydium Komisji.

W posiedzeniu udział wzięli pracownicy Kancelarii Sejmu: Anna Osińska – wicedyrektor Biura Komisji Sejmowych, Magda Jedynak, Anna Ornat i Adrian Konefał – z sekretariatu Komisji w Biurze Komisji Sejmowych.
```

```
Komisja Administracji i Spraw Wewnętrznych, obradująca pod przewodnictwem poseł Marii Małgorzaty Janyskiej (KO), przewodniczącej Komisji, oraz Małgorzaty Pępek (KO), zastępczyni przewodniczącej Komisji, zrealizowała następujący porządek dzienny:

– rozpatrzenie wniosku o powołanie podkomisji stałych;

– informacja ministra Spraw Wewnętrznych i Administracji na temat realizacji ustawy z dnia 18 sierpnia 2011 r. o bezpieczeństwie i ratownictwie w górach na zorganizowanych terenach narciarskich;

– informacja ministra Spraw Wewnętrznych i Administracji na temat najpilniejszych prac legislacyjnych.
```

These sections are not list elements but `<p>` elements. They can be distinguished by the `–` at the beginning of each paragraph. They are also included _expressis verbis_ in the `agenda` property of a corresponding json file. In the json file the html for this section is more structured and could be easier to parse:

```
{"agenda":"<div class=\"agenda-indent-0\">I. Rozpatrzenie Informacji Najwyższej Izby Kontroli o wynikach kontroli \"Obsługa obywateli polskich i cudzoziemców w jednostkach administracji publicznej\"</div>\n<div class=\"agenda-indent-1\">- przedstawia p.o. wicedyrektor Delegatury NIK w Łodzi Przemysław Szewczyk.</div>\n<div class=\"agenda-indent-0\">II. Powołanie podkomisji stałych do spraw:</div>\n<div class=\"agenda-indent-1\">- funkcjonowania zarządzania kryzysowego</div>\n<div class=\"agenda-indent-1\">- obywatelskich, cudzoziemców i migracji</div>\n<div class=\"agenda-indent-1\">- modernizacji i rozwoju Policji, Straży Granicznej, Państwowej Straży Pożarnej i Służby Ochrony Państwa</div>\n<div class=\"agenda-indent-1\">- rozwoju i promocji Ochotniczych Straży Pożarnych.</div>","closed":false,"date":"2024-03-06","num":5,"remote":false,"video":["D872C7820CE0A046C1258AD1003F8E37"]}
```

This list includes names of specific legislative dossiers - it would be a good idea to attach a list of dossiers to each transcript/sitting. These could be later linked to information from an API tracking the legislative process (for example: https://api.sejm.gov.pl/prints.html#lista-drukow).

File numbers are included! For example, the first file mentioned below can be accessed in the following manner: `https://api.sejm.gov.pl/sejm/term7/prints/375`

```
– pierwsze czytanie rządowego projektu ustawy o zmianie ustawy o działach administracji rządowej oraz niektórych innych ustaw (druk nr 375);

– rozpatrzenie poprawki zgłoszonej w czasie drugiego czytania do projektu ustawy o zmianie ustawy o dokumentach paszportowych (druki nr 320 i 394).
```

I don't see a straightforward way to link files with plenary votes.

**Identify speakers in transcripts**

`<p><b>Marszałek Sejmu Elżbieta Witek:</b><br>`
`<p><b>Poseł Jacek Świat (PiS):</b><br>`
`Poseł Izabela Katarzyna Mrzygłocka (KO):`

Looks like the party affiliation of VIPs is not included
I will have to crossreference each bold section with a ful list of representatives for a given term?

Unfortunately there are problematic cases like this one:

`<p><b>Zastępca komendanta głównego Państwowej Straży Pożarnej nadbryg. Krzysztof Hejduk:</b><br>`

Regex and bs4 are not going to be enough. In such cases the only solution I see is to check if we're dealing with a representative first (by crossreferencing with a list of representatives) and if not include the full title (full bold section) to identify a guest. People who appear multiple times but their job title changes will be registered as different speakers. I might consider using some sort of fuzzy matching later.

All speakers are first listed in the Speakers (_Mówcy_) section...

And then we have the _"W posiedzeniu udział wzięli..."_ ("Participated in the sitting") section which is just a `<p>` element (or more than one, always starting with "W posiedzeniu udział...").

```
W posiedzeniu udział wzięli: Maciej Duszczyk i Wiesław Leśniakiewicz podsekretarze stanu w Ministerstwie Spraw Wewnętrznych i Administracji wraz ze współpracownikami, Józef Galica zastępca komendanta głównego Państwowej Straży Pożarnej. Rafał Jankowski przewodniczący Zarządu Głównego Niezależnego Samorządnego Związku Zawodowego Policjantów, Anna Kwasiborska i Joanna Rosińska wiceprzewodniczące Krajowej Komisji Niezależnego Samorządnego Związku Zawodowego Pracowników Policji, Bolesław Pietrzyk prezes Tatrzańskiego Ochotniczego Pogotowia Ratunkowego, Jerzy Płókarz prezes Krajowego Sztabu Ratownictwa Społecznej Sieci Ratunkowej, Krzysztof Sadowski wiceprzewodniczący Rady Krajowej Sekcji Funkcjonariuszy i Pracowników Policji Niezależnego Samorządnego Związku Zawodowego „Solidarność” oraz Jerzy Siodłak naczelnik Górskiego Ochotniczego Pogotowia Ratunkowego wraz ze współpracownikiem.

W posiedzeniu udział wzięli pracownicy Kancelarii Sejmu: Daniel Kędzierski, Anna Majewska i Anna Ornat – z sekretariatu Komisji w Biurze Komisji Sejmowych.
```

Then, _"W posiedzeniu udział wzięli..."_ is followed by the `<p><b>...</b></p>` introducing the first speaker. After that `<p>...</p>` elements with utterances and `<p><b>...</b></p>` elements with speaker names and functions follow.

**Speakers (_Mówcy_) section**

```
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body><h2>Komisje:</h2><ul><li>Komisja Administracji i Spraw Wewnętrznych /nr 2/</li></ul><h2>Mówcy:</h2><ul><li>Zastępca komendanta głównego Policji nadinsp. Dariusz Augustyniak
</li><li>Poseł Zbigniew Bogucki /PiS/
</li><li>Zastępca dyrektora Departamentu Finansowania Sfery Budżetowej w Ministerstwie Finansów Karol Czajkowski
...
```



Based on some random browsing it seems reasonable that there are going to be some inconsistencies in the way the html files with transcripts are structured. My general approach for identifying speakers:

1. Identify `b` tags with speaker names

    - remove all `b` tags starting with "-" (they usually describe items on the agenda)
    - remove all `b` tags staring with "a)", "b)", "1)" etc. (as above)
    - remove all `b` tags that are sprinkled inside larger blocks of text (are not preceded by `p`). These usually mention civil servants present at a sitting who do not participate actively (they don't say anything)

2. Identify representatives by matching them with a list of speakers and, later, the list of all representatives for a given term.

3. Identify guests:

    - some will have correctly formatted `p` tags: `<b>Guest title and name<\b>`
    - some will not. I've seen things like this: `<p><b>speaker title<\b>&nbsp<b>speaker name<\b>` (`&nbsp` = hard space)

    - Possibly helpful regularity: all speaker tags end with ":"

4. There are also "stage directions" and italics in the text. They should be removed. `</p><p><i>[Po przerwie]</i>`

5. HTML is incorrectly formatted for many/most transcripts. It usually has the following structure:

```
<p><b>Speaker tag</b><br> The speaker says something... <p> New paragraph starts but the previous 'p' tag is not closed. <p> And another paragraph starts without closing the previous one.
```
 My general approach is to search for consecutive speaker tags inside the transcript text and get all text _inbetween_. Then, split this text  based on <br> (separates speaker tag and the speaker's first utterance) and <p> tags.

Look through a sample of transcripts and see if there are any other irregularities that have to be accounted for.

In [9]:
transcripts

{'10': {'RRW': {1: '<h2>Komisje:</h2><ul><li>Komisja Rolnictwa i Rozwoju Wsi /nr 1/</li></ul><h2>Mówcy:</h2><ul><li>Poseł Jacek Bogucki /PiS/\r</li><li>Poseł Piotr Borys /KO/\r</li><li>Poseł Mirosław Maliszewski /PSL-TD/\r</li><li>Sekretarz Komisji Dariusz Myrcha\r</li><li>Wicemarszałek Sejmu Dorota Niedziela\r</li><li>Poseł Kazimierz Plocke /KO/\r</li><li>Poseł Jarosław Sachajko /Kukiz15/\r</li><li>Poseł Zbigniew Ziejewski /PSL-TD/</li></ul><section class="transcript">\r\n<p>Komisja Rolnictwa i Rozwoju Wsi, obradująca pod przewodnictwem wicemarszałka Sejmu <b>Doroty Niedzieli</b>, dokonała:\r\n<p><b>– wyboru prezydium Komisji.</b>\r\n<p>W posiedzeniu udział wzięli pracownicy Kancelarii Sejmu: <b>Krzysztof Karkowski</b>&nbsp;– wicedyrektor Sekretariatu Posiedzeń Sejmu, <b>Jolanta Boratyn-Dąbkowska</b>, <b>Dariusz Myrcha</b>,<b>&nbsp;Paulina Przybysz</b>, <b>Anna Rajewska</b>&nbsp;– z sekretariatu Komisji w Biurze Komisji Sejmowych.\r\n<p><b>Wicemarszałek Sejmu Dorota Niedziela:</b><br>

In [10]:
html = transcripts['10']['RRW'][1]

transcript = Transcript(html)

In [11]:
transcript.speakers

['Poseł Jacek Bogucki (PiS)',
 'Poseł Piotr Borys (KO)',
 'Poseł Mirosław Maliszewski (PSL-TD)',
 'Sekretarz Komisji Dariusz Myrcha',
 'Wicemarszałek Sejmu Dorota Niedziela',
 'Poseł Kazimierz Plocke (KO)',
 'Poseł Jarosław Sachajko (Kukiz15)',
 'Poseł Zbigniew Ziejewski (PSL-TD)']

In [12]:
transcript_processed = transcript.separate_speakers_and_statements()

In [13]:
transcript_processed

[(True, 'Wicemarszałek Sejmu Dorota Niedziela'),
 (False,
  'Szanowni państwo, Wysoka Komisjo, pozwólcie, że się przywitam. Po dwunastu latach pracy w tej Komisji chciałabym wszystkich państwa posłów powitać w Komisji Rolnictwa i pogratulować wyboru właśnie tej Komisji. Dziś spotykam się z wami w innej roli, ale bardzo się cieszę. Życzę miłej pracy w nadchodzącej kadencji.'),
 (False,
  'Otwieram zwołane przez marszałka Sejmu na podstawie art. 20 ust. 2 regulaminu Sejmu pierwsze posiedzenie Komisji Rolnictwa i Rozwoju Wsi. Porządek dzisiejszego posiedzenia przewiduje wybór prezydium Komisji, przewodniczącego oraz zastępców przewodniczącego. Zgodnie z art. 163a regulaminu Sejmu uchwały Komisji zapadają większością głosów w obecności co najmniej jednej trzeciej liczby członków Komisji.'),
 (False, 'Na podstawie listy obecności członków Komisji stwierdzam kworum.'),
 (False,
  'Przypominam także, że Komisja na podstawie art. 20 ust. 3 regulaminu Sejmu powołuje i odwołuje członków prezydiu

In [1]:
data = transcript.get_structured_data_from_transcript()

NameError: name 'transcript' is not defined

In [15]:
data

[{'speaker': 'Wicemarszałek Sejmu Dorota Niedziela',
  'utterances': ['Szanowni państwo, Wysoka Komisjo, pozwólcie, że się przywitam. Po dwunastu latach pracy w tej Komisji chciałabym wszystkich państwa posłów powitać w Komisji Rolnictwa i pogratulować wyboru właśnie tej Komisji. Dziś spotykam się z wami w innej roli, ale bardzo się cieszę. Życzę miłej pracy w nadchodzącej kadencji.',
   'Otwieram zwołane przez marszałka Sejmu na podstawie art. 20 ust. 2 regulaminu Sejmu pierwsze posiedzenie Komisji Rolnictwa i Rozwoju Wsi. Porządek dzisiejszego posiedzenia przewiduje wybór prezydium Komisji, przewodniczącego oraz zastępców przewodniczącego. Zgodnie z art. 163a regulaminu Sejmu uchwały Komisji zapadają większością głosów w obecności co najmniej jednej trzeciej liczby członków Komisji.',
   'Na podstawie listy obecności członków Komisji stwierdzam kworum.',
   'Przypominam także, że Komisja na podstawie art. 20 ust. 3 regulaminu Sejmu powołuje i odwołuje członków prezydium w głosowaniu 

In [16]:
# GET MPs

all_mps = []
for  term in range(1, 11):
    url = f"https://api.sejm.gov.pl/sejm/term{term}/MP"
    mps = get_info.get_json_data(url)
    all_mps.append(mps)

In [85]:
all_mps
#len(all_mps)
#get_info.write_json_to_file(all_mps, "data/all_mps.json")

[[],
 [],
 [{'active': False,
   'birthDate': '1952-01-01',
   'birthLocation': 'Lipnica Wielka',
   'club': 'AWS',
   'districtName': 'Nowy Sącz',
   'districtNum': 28,
   'educationLevel': 'wyższe',
   'firstLastName': 'Franciszek Adamczyk',
   'firstName': 'Franciszek',
   'id': 1,
   'lastFirstName': 'Adamczyk Franciszek',
   'lastName': 'Adamczyk',
   'numberOfVotes': 0},
  {'active': False,
   'birthDate': '1968-03-07',
   'birthLocation': 'Ruda Śląska',
   'club': 'AWS',
   'districtName': 'Opole',
   'districtNum': 30,
   'educationLevel': 'wyższe',
   'firstLastName': 'Elżbieta Adamska-Wedler',
   'firstName': 'Elżbieta',
   'id': 2,
   'lastFirstName': 'Adamska-Wedler Elżbieta',
   'lastName': 'Adamska-Wedler',
   'numberOfVotes': 0,
   'secondName': 'Donata'},
  {'active': False,
   'birthDate': '1963-05-23',
   'birthLocation': 'Ruda Śląska',
   'club': 'SLD',
   'districtName': 'Kielce',
   'districtNum': 18,
   'educationLevel': 'średnie zawodowe',
   'firstLastName': 'Wł

In [87]:
# A nested list (mps nested in terms)
all_mps = get_info.read_json_from_file("data/all_mps.json")

In [88]:
#all_mps
for term in all_mps:
    #if term:
    for mp in term:
        mp["birth_and_name"] = mp.get("birthDate", "9999") + "_" + mp.get("firstLastName", "9999")

In the final version of the app I should consider some form of fuzzy matching on MP names to account for the possibility that some female MPs changed their names.

Genearal architecture of the db:
- the term field should be linked to the MP field
- every MP should have a unique ID independent of the term

In [115]:
for term in range(1, len(all_mps)+1):
    for mp in all_mps[term-1]:
        mp["term"] = term
    

In [116]:
all_mps_flat = [mp for term in all_mps for mp in term]


In [117]:
list(filter(lambda x: x["term"] == 10, all_mps_flat))

[{'active': True,
  'birthDate': '1959-01-04',
  'birthLocation': 'Krzeszowice',
  'club': 'PiS',
  'districtName': 'Kraków',
  'districtNum': 13,
  'educationLevel': 'wyższe',
  'email': 'Andrzej.Adamczyk@sejm.pl',
  'firstLastName': 'Andrzej Adamczyk',
  'firstName': 'Andrzej',
  'id': 1,
  'lastFirstName': 'Adamczyk Andrzej',
  'lastName': 'Adamczyk',
  'numberOfVotes': 45171,
  'profession': 'ekonomista',
  'secondName': 'Mieczysław',
  'voivodeship': 'małopolskie',
  'birth_and_name': '1959-01-04_Andrzej Adamczyk',
  'term': 10},
 {'active': True,
  'birthDate': '1961-06-26',
  'birthLocation': 'Elbląg',
  'club': 'KO',
  'districtName': 'Gdańsk',
  'districtNum': 25,
  'educationLevel': 'średnie ogólne',
  'email': 'Piotr.Adamowicz@sejm.pl',
  'firstLastName': 'Piotr Adamowicz',
  'firstName': 'Piotr',
  'id': 2,
  'lastFirstName': 'Adamowicz Piotr',
  'lastName': 'Adamowicz',
  'numberOfVotes': 23147,
  'profession': 'dziennikarz',
  'voivodeship': 'pomorskie',
  'birth_and_name

In [118]:
get_info.write_json_to_file(all_mps_flat, "data/all_mps_flat.json")

In [82]:
df = pd.DataFrame.from_dict(all_mps_flat)

In [95]:
df.columns

Index(['active', 'birthDate', 'birthLocation', 'club', 'districtName',
       'districtNum', 'educationLevel', 'firstLastName', 'firstName', 'id',
       'lastFirstName', 'lastName', 'numberOfVotes', 'birth_and_name',
       'secondName', 'inactiveCause', 'voivodeship', 'waiverDesc',
       'profession', 'email'],
      dtype='object')

In [93]:
print(df.birth_and_name.unique().__len__()) # 2025

2025


In [103]:
# Get unique MPs
unique_mps = df[["firstLastName", "birthDate"]].drop_duplicates()

In [104]:
unique_mps


,firstLastName,birthDate
0,Franciszek Adamczyk,1952-01-01
1,Elżbieta Adamska-Wedler,1968-03-07
2,Władysław Adamski,1963-05-23
3,Romuald Ajchler,1949-01-19
4,Andrzej Anusz,1965-08-27
...,...,...
3922,Wojciech Michał Zubowski,1979-11-12
3926,Anna Baluch,1973-06-03
3929,Alicja Łuczak,1961-06-17
3930,Dominik Jaśkowiec,1977-06-12


In [105]:

get_info.write_json_to_file(unique_mps.to_dict('records'), "data/unique_mps.json")

In [83]:
# Count the number of MPs per term and arrange them by count
summary = df.groupby(["term", "birth_and_name"]).size().reset_index(name="count").sort_values(by=["term", "count"], ascending=False)
summary['count'].max()

KeyError: 'term'

In [114]:
df

# Check if there are rows with the same value in the birth_and_name column

,active,birthDate,birthLocation,club,districtName,districtNum,educationLevel,firstLastName,firstName,id,lastFirstName,lastName,numberOfVotes,birth_and_name,secondName,inactiveCause,voivodeship,waiverDesc,profession,email
0,False,1952-01-01,Lipnica Wielka,AWS,Nowy Sącz,28,wyższe,Franciszek Adamczyk,Franciszek,1,Adamczyk Franciszek,Adamczyk,0,1952-01-01_Franciszek Adamczyk,NaN,NaN,NaN,NaN,NaN,NaN
1,False,1968-03-07,Ruda Śląska,AWS,Opole,30,wyższe,Elżbieta Adamska-Wedler,Elżbieta,2,Adamska-Wedler Elżbieta,Adamska-Wedler,0,1968-03-07_Elżbieta Adamska-Wedler,Donata,NaN,NaN,NaN,NaN,NaN
2,False,1963-05-23,Ruda Śląska,SLD,Kielce,18,średnie zawodowe,Władysław Adamski,Władysław,3,Adamski Władysław,Adamski,0,1963-05-23_Władysław Adamski,Roman,NaN,NaN,NaN,NaN,NaN
3,False,1949-01-19,Duszniki Wlkp.,SLD,Piła,32,średnie zawodowe,Romuald Ajchler,Romuald,4,Ajchler Romuald,Ajchler,0,1949-01-19_Romuald Ajchler,Kazimierz,NaN,NaN,NaN,NaN,NaN
4,False,1965-08-27,Warszawa,AWS,Warszawa,2,wyższe,Andrzej Anusz,Andrzej,5,Anusz Andrzej,Anusz,0,1965-08-27_Andrzej Anusz,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,True,1983-06-15,Ryki,niez.,Chełm,7,wyższe,Monika Pawłowska,Monika,463,Pawłowska Monika,Pawłowska,10789,1983-06-15_Monika Pawłowska,Jolanta,NaN,lubelskie,NaN,specjalista do spraw stosunków międzynarodowych,Monika.Pawlowska@sejm.pl
3929,True,1961-06-17,Stawiszyn,KO,Kalisz,36,wyższe,Alicja Łuczak,Alicja,464,Łuczak Alicja,Łuczak,5186,1961-06-17_Alicja Łuczak,NaN,NaN,wielkopolskie,NaN,nauczycielka,Alicja.Luczak@sejm.pl
3930,True,1977-06-12,Kraków,KO,Kraków,13,wyższe,Dominik Jaśkowiec,Dominik,465,Jaśkowiec Dominik,Jaśkowiec,6093,1977-06-12_Dominik Jaśkowiec,Andrzej,NaN,małopolskie,NaN,nauczyciel akademicki,Dominik.Jaskowiec@sejm.pl
3931,True,1980-07-07,Głubczyce,KO,Opole,21,wyższe,Paweł Masełko,Paweł,466,Masełko Paweł,Masełko,8322,1980-07-07_Paweł Masełko,Jan,NaN,opolskie,NaN,nauczyciel,Pawel.Maselko@sejm.pl


In [34]:
df.columns

Index(['active', 'birthDate', 'birthLocation', 'club', 'districtName',
       'districtNum', 'educationLevel', 'firstLastName', 'firstName', 'id',
       'lastFirstName', 'lastName', 'numberOfVotes', 'birth_and_name',
       'birth_and_loc', 'term ', 'secondName', 'inactiveCause', 'voivodeship',
       'waiverDesc', 'profession', 'email'],
      dtype='object')

In [66]:
# Group by full name and date of birth
grouped = df.groupby(['firstLastName', 'birthDate'])

# Find groups with more than one unique term specific ID
potential_duplicates = grouped.filter(lambda x: x['id'].nunique() > 1)

# Remove MPs who appear to be the same across multiple terms
# Assuming the same person will have different term_specific_ids but same full name and DOB across terms
duplicates = potential_duplicates.groupby(['firstLastName', 'birthDate']).filter(lambda x: len(x['term'].unique()) == 1)

# Display potential duplicates that need manual verification
if not duplicates.empty:
    print("Potential duplicate MPs found:")
    print(duplicates)
else:
    print("No duplicate MPs found.")

No duplicate MPs found.


In [142]:
sittings = get_info.read_json_from_file("data/sittings_info.json")

In [143]:
sittings

for term in sittings:
    term = Term.objects.get(number = int(term))

{'9': {'EPS': [{'agenda': '<div class="agenda-indent-0">Wybór przewodniczącego i zastępcy przewodniczącego Komisji.</div>',
    'closed': False,
    'date': '2019-11-19',
    'num': 1,
    'remote': False,
    'video': [],
    'code': 'EPS',
    'term': 9},
   {'agenda': '<div class="agenda-indent-0">I. Rozpatrzenie projektu planu pracy Komisji Etyki Poselskiej na okres od 1 stycznia do 31 lipca 2020 r.</div>\n<div class="agenda-indent-1">- przedstawia Przewodniczący Komisji.</div>\n<div class="agenda-indent-0">II. Sprawy bieżące.</div>',
    'closed': False,
    'date': '2019-12-19',
    'num': 2,
    'remote': False,
    'video': []},
   {'agenda': '<div class="agenda-indent-0">I. Rozpatrzenie projektu uwag Komisji Etyki Poselskiej dotyczących wypełniania oświadczeń o stanie majątkowym składanych przez posłów za 2019 rok.</div>\n<div class="agenda-indent-0">II. Zajęcie stanowiska w sprawach, które wpłynęły do Komisji.</div>',
    'closed': False,
    'date': '2020-02-12',
    'num': 